# Third prompt

長文は要約して出力した結果に対し、3つ目のプロンプトを追加し、コンテキストを元に文を作成できているかを判定する。

In [1]:
import os
import pandas as pd
import openai
import datetime
import tiktoken
from sudachipy import tokenizer
from sudachipy import dictionary
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from typing import List

from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from pydantic import BaseModel, Field

In [2]:
# OpenAI APIキーを設定
openai.api_key = input()

In [3]:
# OpenAIの言語モデルを設定（ここではGPT-3を使用）
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai.api_key)

C:\Users\tatsu\AppData\Local\Temp\ipykernel_38828\1591884372.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai.api_key)


## 回答ファイルを読み込む

In [4]:
# 提供されたCSVファイルを読み込み
query_df = pd.read_csv("../submit/20241005-0121_predictions.csv", encoding="utf-8")

In [6]:
query_df.head()

,index,problem,full_answer,answer,evidence
0,1,競漕会の三日前のレースコースでの結果は、農科と文科でどれくらいの秒数差があったか？,5秒の差がありました。,5秒の差がありました。,三 競漕の日はだんだん近づいて来る。 その一週間ほど前に学習院の競漕会があった。 それには文...
1,2,骸骨男の正体は誰ですか？作中で言及されている氏名で答えること。,笠原さんです。,笠原さんです。,「アッ、骸骨……。」 それは、あの骸骨男でした。 いつも着ているオーバーや洋服をぬいで、はだ...
2,3,骸骨男はバスの中に足跡を一切残さずにどうやって抜け出しましたか？,骸骨男は、バスの床板にあったばね仕掛けの落とし穴を使って抜け出しました。,骸骨男は、バスの床板にあったばね仕掛けの落とし穴を使って抜け出しました。,とつぶやいています。 「骸骨男がいなくなったのですか。」 井上君が、たずねます。 「うん、い...
3,4,殺人罪で裁判にかけられた兄が登場しますが、その理由は何ですか？,若い頃、兄は恋人と心中を図り、彼女を短刀で殺害したためです。,若い頃、兄は恋人と心中を図り、彼女を短刀で殺害したためです。,そして、仏様に捧げる祈りの文句を繰り返した。 木魚の音や読経は数時間続いた。 それらが終わっ...
4,5,小説「芽生」で出てくる国名は何種類ですか？,分かりません。,分かりません。,十の一 四月あまり過ぎたり。 霜に染みたる南天の影長々と庭に臥す午後四時過ぎ、相も変わらず肥...


In [7]:
query_df = query_df.rename(columns={"answer": "tmp_answer"})

In [8]:
query_df.columns

Index(['index', 'problem', 'full_answer', 'tmp_answer', 'evidence'], dtype='object')

In [20]:
# LLMを使って文章の修正を行う関数
def check_answer(problem: str, tmp_answer: str, evidence: str) -> str:

    check_prompt = PromptTemplate(
        input_variables=["problem", "full_answer", "evidence"],
        template=
            """あなたは回答の確からしさを判定する優れたチェックシステムです。
            問題に対し、仮の回答とその根拠が与えられています。
            仮の回答が根拠から導き出せていない場合、「分かりません。」と回答してください。
            それ以外の場合、仮の回答を1文字も変えずそのまま回答してください。

                f"問題: {problem}\n\n"
                f"仮の回答: {tmp_answer}\n"
                f"根拠: {evidence}\n"
            回答:"""
    )
    chain = check_prompt | llm

    response = chain.invoke(
        {"problem": problem, "tmp_answer": tmp_answer, "evidence": evidence}
    )
    return response.content

In [21]:
def check_and_summarize_answers(query_df: pd.DataFrame) -> pd.DataFrame:
    # "answer" 列に対して処理を適用
    query_df["answer"] = query_df["tmp_answer"]
    for i in range(len(query_df.index)):
        print(i)
        query_df["answer"][i] = check_answer(query_df["problem"][i], query_df["tmp_answer"][i], query_df["evidence"][i])
        print(query_df["answer"][i])
    return query_df

In [22]:
query_df = check_and_summarize_answers(query_df)

0


C:\Users\tatsu\AppData\Local\Temp\ipykernel_38828\1369155956.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_df["answer"][i] = check_answer(query_df["problem"][i], query_df["tmp_answer"][i], query_df["evidence"][i])


分かりません。
1


C:\Users\tatsu\AppData\Local\Temp\ipykernel_38828\1369155956.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_df["answer"][i] = check_answer(query_df["problem"][i], query_df["tmp_answer"][i], query_df["evidence"][i])


分かりません。
2


...

C:\Users\tatsu\AppData\Local\Temp\ipykernel_38828\1369155956.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_df["answer"][i] = check_answer(query_df["problem"][i], query_df["tmp_answer"][i], query_df["evidence"][i])


分かりません。
58


C:\Users\tatsu\AppData\Local\Temp\ipykernel_38828\1369155956.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_df["answer"][i] = check_answer(query_df["problem"][i], query_df["tmp_answer"][i], query_df["evidence"][i])


分かりません。
59
分かりません。


C:\Users\tatsu\AppData\Local\Temp\ipykernel_38828\1369155956.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_df["answer"][i] = check_answer(query_df["problem"][i], query_df["tmp_answer"][i], query_df["evidence"][i])


In [14]:
query_df.head(20)

,index,problem,full_answer,tmp_answer,evidence,answer
0,1,競漕会の三日前のレースコースでの結果は、農科と文科でどれくらいの秒数差があったか？,5秒の差がありました。,5秒の差がありました。,三 競漕の日はだんだん近づいて来る。 その一週間ほど前に学習院の競漕会があった。 それには文...,分かりません。
1,2,骸骨男の正体は誰ですか？作中で言及されている氏名で答えること。,笠原さんです。,笠原さんです。,「アッ、骸骨……。」 それは、あの骸骨男でした。 いつも着ているオーバーや洋服をぬいで、はだ...,分かりません。
2,3,骸骨男はバスの中に足跡を一切残さずにどうやって抜け出しましたか？,骸骨男は、バスの床板にあったばね仕掛けの落とし穴を使って抜け出しました。,骸骨男は、バスの床板にあったばね仕掛けの落とし穴を使って抜け出しました。,とつぶやいています。 「骸骨男がいなくなったのですか。」 井上君が、たずねます。 「うん、い...,骸骨男は、バスの床板にあったばね仕掛けの落とし穴を使って抜け出しました。
3,4,殺人罪で裁判にかけられた兄が登場しますが、その理由は何ですか？,若い頃、兄は恋人と心中を図り、彼女を短刀で殺害したためです。,若い頃、兄は恋人と心中を図り、彼女を短刀で殺害したためです。,そして、仏様に捧げる祈りの文句を繰り返した。 木魚の音や読経は数時間続いた。 それらが終わっ...,分かりません。
4,5,小説「芽生」で出てくる国名は何種類ですか？,分かりません。,分かりません。,十の一 四月あまり過ぎたり。 霜に染みたる南天の影長々と庭に臥す午後四時過ぎ、相も変わらず肥...,分かりません。
5,6,武男と浪子のうち、どちらが野外での楽しみをより積極的に表現していますか？,武男がより積極的に野外での楽しみを表現しています。,武男がより積極的に野外での楽しみを表現しています。,「僕は一人で歩く方が勝手だ」 これほど手強く打ち払えばいかなる強敵も退散すべしと思いきや、な...,分かりません。
6,7,小説「芽生」で、主人公が東京から小諸に帰る時期から、実際に引越しが完了するまでの間はだいたい...,分かりません。,分かりません。,「いいえ、ちょっとも疲れはいたしません。どうかお話し遊ばして」 茶を入れかえて、幾は次に立ち...,分かりません。
7,8,「競漕」の中で、文科のボート部メンバーが実際に酒を飲むシーンは何回か？,1回です。,1回です。,そしていつもよりは活気づいて艇庫に船を蔵めた。 夕飯には褒賞の意味で窪田が特別に一人約二合ほ...,分かりません。
8,9,文中に登場するカタカナ表記された北海道の地名を全て挙げてください。,分かりません。,分かりません。,海は実に凪げるなり。 近午の空は天心にいたるまで蒼々と晴れて雲なく、一碧の海は所々練れるよう...,分かりません。
9,10,兄が情死を試みた時点から、実際に死亡するまでおよそ何年経過したと考えられますか？,約七年が経過したと考えられます。,約七年が経過したと考えられます。,そして、仏様に捧げる祈りの文句を繰り返した。 木魚の音や読経は数時間続いた。 それらが終わっ...,分かりません。


In [44]:
# 必要な列（id, answer, evidence）をヘッダなしでCSVに書き出し
query_df[['index', 'answer', 'evidence']].to_csv(
    "../submit/predictions.csv",
    index=False,
    header=False,
    encoding="utf-8-sig"
)

In [45]:
# backup
dt_now = datetime.datetime.now()
ymdm = dt_now.strftime("%Y%m%d-%H%M")

query_df[['index', 'problem', 'full_answer', 'answer', 'evidence']].to_csv(
    f"../submit/{ymdm}_predictions.csv",
    index=False,
    header=True,
    encoding="utf-8-sig"
)